# Name Yash Parihar
# Roll No. 19337

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from tqdm import tqdm
from torchvision.models.resnet import ResNet18_Weights
from torchvision.models import vgg16, resnet50, resnet101
import timm

/Users/yashparihar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, Subset

class ModelTransforms:
    def __init__(self):
        pass

    def get_transform(self, model_name):
        if model_name.lower() == "resnet18":
            return transforms.Compose([
                transforms.Resize((32, 32)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])
        elif model_name.lower() == "alexnet":
            return transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])
        elif model_name.lower() == "vgg16":
            return transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        elif model_name.lower() == "resnet50" or model_name.lower() == "resnet101":
            return transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        else:
            raise ValueError(f"Unsupported model: {model_name}")

def get_data_loaders(model_name, train_subset_size=0.25, batch_size=64):
    # Get the appropriate transform for the model
    transforms_handler = ModelTransforms()
    transform = transforms_handler.get_transform(model_name)
    
    # Define the full training dataset
    train_data_full = datasets.SVHN(root='./data', split='train', download=False, transform=transform)
    
    # Determine the subset size based on the percentage provided
    subset_size = int(train_subset_size * len(train_data_full))
    
    # Create a subset of the training data
    train_data_subset = Subset(train_data_full, list(range(subset_size)))
    
    # Define the data loader for the training subset
    train_loader_subset = DataLoader(train_data_subset, batch_size=batch_size, shuffle=True)
    
    # Define the testing dataset
    test_data = datasets.SVHN(root='./data', split='test', download=False, transform=transform)
    
    # Define the data loader for the testing dataset
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return train_loader_subset, test_loader

In [11]:
%%time
# Example usage:
model_name = "resnet18"  # Change this to the model you are using
train_loader_subset, test_loader = get_data_loaders(model_name)

# Defing the model
model = models.resnet18(weights=ResNet18_Weights.DEFAULT) # using latest weights
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for SVHN

# Moving the model to the GPU 
device = torch.device("mps")
model.to(device)

criterion = nn.CrossEntropyLoss() # Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001) # Optimizer

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Using tqdm to create a progress bar
    with tqdm(train_loader_subset, unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
            t.set_postfix(loss=running_loss/len(train_loader_subset.dataset))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader_subset.dataset)}")

# Evaluating the loop
model.eval()
correct = 0
total = 0

# Using tqdm to create a progress bar
with tqdm(test_loader, unit="batch") as t:
    for inputs, labels in t:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")

100%|███████████████████████████| 287/287 [00:15<00:00, 18.46batch/s, loss=0.89]


Epoch 1/30, Train Loss: 0.8899229135369583


100%|███████████████████████████| 287/287 [00:14<00:00, 19.66batch/s, loss=0.48]


Epoch 2/30, Train Loss: 0.4797538500549726


100%|██████████████████████████| 287/287 [00:14<00:00, 19.66batch/s, loss=0.356]


...

Epoch 29/30, Train Loss: 0.023402826952387656


100%|█████████████████████████| 287/287 [00:31<00:00,  9.22batch/s, loss=0.0625]


Epoch 30/30, Train Loss: 0.06245266073780553


100%|██████████████████████████████████████| 407/407 [00:09<00:00, 41.12batch/s]

Test Accuracy: 0.9007375537799631
CPU times: user 8min 16s, sys: 36.6 s, total: 8min 52s
Wall time: 15min 3s


In [14]:
%%time
model_name = "alexnet"  
train_loader_subset, test_loader = get_data_loaders(model_name)

model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT) # latest weights
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 10)  # 10 classes for SVHN

device = torch.device("mps")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    with tqdm(train_loader_subset, unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
            t.set_postfix(loss=running_loss/len(train_loader_subset.dataset))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader_subset.dataset)}")

model.eval()
correct = 0
total = 0

with tqdm(test_loader, unit="batch") as t:
    for inputs, labels in t:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")

100%|███████████████████████████| 287/287 [03:07<00:00,  1.53batch/s, loss=2.25]


Epoch 1/10, Train Loss: 2.2528395861325414


100%|███████████████████████████| 287/287 [03:07<00:00,  1.53batch/s, loss=2.24]


Epoch 2/10, Train Loss: 2.23847741222517


100%|███████████████████████████| 287/287 [03:06<00:00,  1.54batch/s, loss=2.24]


...

Epoch 9/10, Train Loss: 2.2373330322587233


100%|███████████████████████████| 287/287 [01:30<00:00,  3.18batch/s, loss=2.24]


Epoch 10/10, Train Loss: 2.2364374550328554


100%|██████████████████████████████████████| 407/407 [01:38<00:00,  4.13batch/s]

Test Accuracy: 0.1958743085433313
CPU times: user 23min 30s, sys: 55min 47s, total: 1h 19min 17s
Wall time: 49min 41s


In [16]:
%%time
model_name = "vgg16"  
train_loader_subset, test_loader = get_data_loaders(model_name)

model = vgg16(weights=models.VGG16_Weights.DEFAULT) # latest weights

device = torch.device("mps")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
        
    with tqdm(train_loader_subset, unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
            t.set_postfix(loss=running_loss/len(train_loader_subset.dataset))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader_subset.dataset)}")


model.eval()
correct = 0
total = 0

with tqdm(test_loader, unit="batch") as t:
    for inputs, labels in t:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")

100%|███████████████████████████| 287/287 [02:05<00:00,  2.28batch/s, loss=1.95]


Epoch 1/2, Train Loss: 1.9476095129142517


100%|██████████████████████████| 287/287 [05:23<00:00,  1.13s/batch, loss=0.887]


Epoch 2/2, Train Loss: 0.8868446732121966


100%|██████████████████████████████████████| 407/407 [00:25<00:00, 15.92batch/s]

Test Accuracy: 0.759219422249539
CPU times: user 29.8 s, sys: 3min 13s, total: 3min 42s
Wall time: 7min 57s


In [3]:
%%time
model_name = "resnet50"  
train_loader_subset, test_loader = get_data_loaders(model_name)

model = resnet50(weights=models.ResNet50_Weights.DEFAULT) # latest weights
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for SVHN

device = torch.device("mps")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
     
    with tqdm(train_loader_subset, unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
            t.set_postfix(loss=running_loss/len(train_loader_subset.dataset))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader_subset.dataset)}")

model.eval()
correct = 0
total = 0

with tqdm(test_loader, unit="batch") as t:
    for inputs, labels in t:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")

100%|██████████████████████████| 287/287 [14:09<00:00,  2.96s/batch, loss=0.587]


Epoch 1/3, Train Loss: 0.5868056449838583


100%|████████████████████████████| 287/287 [16:31<00:00,  3.45s/batch, loss=0.3]


Epoch 2/3, Train Loss: 0.3002523713385849


100%|██████████████████████████| 287/287 [14:53<00:00,  3.11s/batch, loss=0.242]


Epoch 3/3, Train Loss: 0.24185254095689535


100%|██████████████████████████████████████| 407/407 [09:20<00:00,  1.38s/batch]

Test Accuracy: 0.927358635525507
CPU times: user 10min 37s, sys: 25min 29s, total: 36min 7s
Wall time: 54min 56s


In [5]:
%%time
model_name = "resnet101"  
train_loader_subset, test_loader = get_data_loaders(model_name)

model = resnet101(weights=models.ResNet101_Weights.DEFAULT) # latest weights
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # 10 classes for SVHN

device = torch.device("mps")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    with tqdm(train_loader_subset, unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            
            t.set_postfix(loss=running_loss/len(train_loader_subset.dataset))
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader_subset.dataset)}")

model.eval()
correct = 0
total = 0

with tqdm(test_loader, unit="batch") as t:
    for inputs, labels in t:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {correct / total}")

  1%|▏                        | 2/287 [02:27<5:50:38, 73.82s/batch, loss=0.0159]


KeyboardInterrupt: 